In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.embeddings.text_embeddings_inference import TextEmbeddingsInference
from llama_index.core import VectorStoreIndex, Settings, ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from duckduckgo_search import DDGS
from llama_index.readers.web import SimpleWebPageReader, BeautifulSoupWebReader
from llama_index.core.node_parser import HTMLNodeParser, SimpleNodeParser
from llama_index.core import StorageContext

from fastapi import HTTPException
import weaviate
import os
import logging
import yaml

import pandas as pd

/home/brianda/projects/chatbots/dev_chatbot_assistant_for_citizen_assemblies/orchestator-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.basicConfig(level=logging.INFO)


In [3]:
logging.info('This is a info message')
logging.debug('This is a info message')

INFO:root:This is a info message


In [4]:
def disable_logging_during_tests():
    # Store the current log level to restore it later
    original_log_level = logging.getLogger().getEffectiveLevel()

    # Set the log level to a higher level, e.g., WARNING or CRITICAL
    logging.disable(logging.CRITICAL)

    # Run your tests here

    # Restore the original log level after the tests
    logging.disable(original_log_level)

# Call this function before running your tests
#disable_logging_during_tests()

In [37]:
#pip install -U 'protobuf==5.26.1'
#pip install arize-phoenix[evals]

In [38]:
pip freeze | grep duckduckgo


duckduckgo_search==6.1.1
llama-index-tools-duckduckgo==0.1.1
Note: you may need to restart the kernel to use updated packages.


### Observability with Phoenix

**Arize Phoenix**
*AI Observability and Evaluation*

Phoenix is an open-source observability library designed for experimentation, evaluation, and troubleshooting. It allows AI Engineers and Data Scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.

Source: [Arize Phoenix website](https://docs.arize.com/phoenix)

In [39]:
import pandas as pd
import phoenix as px

# Phoenix is
# To view traces in Phoenix, it's needed first have to start a Phoenix server
session = px.launch_app()

/home/brianda/projects/chatbots/dev_chatbot_assistant_for_citizen_assemblies/orchestator-venv/lib/python3.10/site-packages/phoenix/server/app.py:172: DeprecationWarning: The `graphiql` argument is deprecated in favor of `graphql_ide`
  super().__init__(schema, graphiql=graphiql)


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Enable Phoenix tracing via LlamaIndexInstrumentor. Phoenix uses OpenInference traces - an open-source standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [40]:
# Configure orchestator to send traces to Phoenix.
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import SpanLimits, TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

In [41]:
#pip install -U 'llama-index-core==0.10.43'

In [42]:
endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider(span_limits=SpanLimits(max_attributes=100_000))
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

---

### Parámetros y funciones de configuración

In [59]:
from dotenv import load_dotenv
dotenv_path = '../config.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [60]:
EMBED_MODEL_NAME = os.getenv('EMBED_MODEL_NAME')
EMBED_BASE_URL = os.getenv('EMBED_BASE_URL')
EMBED_TIMEOUT = int(os.getenv('EMBED_TIMEOUT'))
EMBED_BATCH_SIZE = int(os.getenv('EMBED_BATCH_SIZE'))
LLM_MODEL_NAME = os.getenv('LLM_MODEL_NAME')
LLM_KEEP_ALIVE = int(os.getenv('LLM_KEEP_ALIVE'))
LLM_REQUEST_TIMEOUT = int(os.getenv('LLM_REQUEST_TIMEOUT'))
LLM_TEMPERATURE = float(os.getenv('LLM_TEMPERATURE'))
LLM_BASE_URL = os.getenv('LLM_BASE_URL')

WEAVIATE_HOST = os.getenv('WEAVIATE_HOST')
WEAVIATE_PORT = '8090' #os.getenv('WEAVIATE_PORT')
WEAVIATE_GRPC_PORT = '50061' #os.getenv('WEAVIATE_GRPC_PORT')
WEAVIATE_URL = os.getenv('WEAVIATE_URL')

INDEX_NAME = os.getenv('INDEX_NAME')
INDEX_NAME_TEMP = os.getenv('INDEX_NAME_TEMP')

CHUNK_SIZE = 1024 #int(os.getenv('CHUNK_SIZE'))
CHUNK_OVERLAP = int(os.getenv('CHUNK_OVERLAP'))

In [61]:
# Function to set up the embeddings model
def setup_embedding_model():
    return TextEmbeddingsInference(
        model_name=EMBED_MODEL_NAME,
        base_url=EMBED_BASE_URL,
        timeout=EMBED_TIMEOUT, 
        embed_batch_size=EMBED_BATCH_SIZE
    )

# Function to set up the LLM model
def setup_llm_model():
    return Ollama(
        model=LLM_MODEL_NAME, 
        keep_alive=LLM_KEEP_ALIVE, 
        request_timeout=LLM_REQUEST_TIMEOUT, 
        temperature=LLM_TEMPERATURE, 
        base_url=LLM_BASE_URL
    )

# Function to configure settings for LLM and embedding model
def configure_settings(llm_model, embed_model):
    Settings.llm = llm_model
    Settings.embed_model = embed_model

def load_custom(code):
    file_path = f'custom_{code}.yml'
    with open(file_path, 'r', encoding='utf-8') as file:
        templates = yaml.safe_load(file)
    return templates

In [62]:
# Get the templates from the YAML file
custom_templates = load_custom('ES_01')
custom_templates

{'QA_PROMPT_STR': '### Contexto de la Información:\nLa información del contexto está abajo.\n---------------------\n{context_str}\n---------------------\nSolo con información del contexto, y no con conocimiento previo, responde a la pregunta **en el mismo idioma** que la pregunta del usuario: {query_str}\n',
 'BASIC_CONTENT_DOCS': "Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.\n\n### Instrucciones Generales:\n1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas.\n2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo.\n3. **Claridad y Concisión:** Responde de forma sintética, concisa y coherente en el mismo idioma que la pregunta.\n4. **Reconducción de Pregunt

In [63]:
# List of sites to which we limit the search, from the YAML file
sites = custom_templates['SITES']
print(sites)

['aemetblog.es', 'eea.europa.eu/es/', 'climate.ec.europa.eu/', 'ipcc.ch']


In [64]:
# Function to connect to Weaviate
def connect_to_weaviate():
    logging.info(f"Iniciando la conexión a weaviate: {WEAVIATE_URL}")
    # It's necessary to indicate the grpc data especially if in docker compose it's redirected from a different port
    weaviate_client = weaviate.connect_to_custom(
                http_host=WEAVIATE_HOST,
                http_port=WEAVIATE_PORT,
                http_secure=False,
                grpc_host= 'localhost',
                grpc_port=WEAVIATE_GRPC_PORT,
                grpc_secure=False,
            )
    logging.info(f"Conectado a weaviate")
    return weaviate_client
    

In [65]:
# Function to initialize the query engine for Chat+RAG from Weaviate documents
def initialize_query_engine_old(weaviate_client, index_name, text_key="text"):
    vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                       index_name=index_name,
                                       text_key=text_key)
    index = VectorStoreIndex.from_vector_store(vector_store)

    # Custom Basic Content
    basic_content = custom_templates['BASIC_CONTENT_DOCS']
    
    qa_prompt_str = custom_templates['QA_PROMPT_STR']

    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=basic_content,
        ),
        ChatMessage(role=MessageRole.SYSTEM, content=qa_prompt_str),
    ]
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

    ## Custom Refine Prompt
    #refine_prompt_str = custom_templates['REFINE_PROMPT_STR']
    #
    #chat_refine_msgs = [
    #    ChatMessage(
    #        role=MessageRole.SYSTEM,
    #        content=refine_prompt_str,
    #    ),
    #    ChatMessage(role=MessageRole.USER, content=refine_prompt_str),
    #]
    #refine_template = ChatPromptTemplate(chat_refine_msgs)

    # With chat_mode='context', for each chat interaction:
    # - first retrieve text from the index using the user message
    # - set the retrieved text as context in the system prompt
    # - return an answer to the user message
    return index.as_query_engine(#chat_mode="context",
                                 text_qa_template=text_qa_template,
                                 #refine_template=refine_template,
                                 similarity_top_k=5
                                 ) 


In [72]:
# Function to initialize the query engine for Chat+RAG from Weaviate documents
# Using hybrid mode
def initialize_query_engine(weaviate_client, index_name, text_key="text", hybrid_mode=False):
    vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                       index_name=index_name,
                                       text_key=text_key)
    index = VectorStoreIndex.from_vector_store(vector_store)

    # Custom Basic Content
    basic_content = custom_templates['BASIC_CONTENT_DOCS']
    
    qa_prompt_str = custom_templates['QA_PROMPT_STR']

    chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=basic_content,
        ),
        ChatMessage(role=MessageRole.SYSTEM, content=qa_prompt_str),
    ]
    text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

    if not hybrid_mode:
        return index.as_query_engine(text_qa_template=text_qa_template,
                                     similarity_top_k=5,
                                     )
    # Query Index with hybrid search
    # (alpha = 0 -> bm25, alpha=1 -> vector search)
    if hybrid_mode:
        return index.as_query_engine(text_qa_template=text_qa_template,
                                    vector_store_query_mode='hybrid',
                                    similarity_top_k=3,
                                    alpha=0.5,
                                    ) 


In [67]:
## Query construction with OR blocks
#def build_duckduckgo_query(query, sites):
#    blocks = [f"({query} site:{site})" for site in sites]
#    duckduckgo_query = " OR ".join(blocks)
#    return duckduckgo_query

### Pruebas RAG Weaviate - documents & Web. Proceso principal y funciones ❎

In [68]:
# Get the keywords from the LLM model for a given query, to use in a web search
def get_keywords_from_llm(query, llm_model):
    messages = [
        ChatMessage(role="system", content="""
                    You are designed to extract the relevant terms to perform a web search from the user query.
                    Total terms must be simple and not exceed 5 words.
                    You must respond only with the search terms separated by spaces,
                    in the same language as the user query.
                    """),
        ChatMessage(role="user", content='¿Qué son las estelas que dejan los aviones en el cielo?'),
        ChatMessage(role="assistant", content="estelas aviones cielo"),
        ChatMessage(role="user", content='¿Qué son las macrogranjas?'),
        ChatMessage(role="assistant", content="macrogranjas"),
    ]
    messages.append(ChatMessage(role="user", content=query))
    keywords = llm_model.chat(messages).message.content
    return keywords

# Search function with DuckDuckGo, using the query constructed with the OR blocks
def search_with_site_filter(keywords, sites, max_results=4):
    site_query = " OR ".join([f"({keywords} site:{site})" for site in sites])
    params = {
        "keywords": site_query,
        "region": '',
        "max_results": max_results,
    }
    with DDGS() as ddg:
        results = list(ddg.text(**params))
    return results

# Function to load documents from URLs
def load_documents_from_urls(urls,html_to_text):
    unique_urls = list(set(urls))  # Remove duplicate URLs
    if html_to_text:
        # Use BeautifulSoupWebReader to load data
        loader = BeautifulSoupWebReader()
        documents = loader.load_data(urls=unique_urls)
    else:
        # Not in use, but just in case
        documents = SimpleWebPageReader(html_to_text=html_to_text).load_data(unique_urls)
    return documents

# Function to set up vector store and storage context
def setup_vector_store_and_context(weaviate_client, index_name, embed_model):
    if weaviate_client.collections.exists(index_name):
        print(f"Eliminando la colección existente: {index_name}")
        weaviate_client.collections.delete(index_name)
    print(f"Configurando y construyendo el índice: {index_name}")
    vector_store = WeaviateVectorStore(weaviate_client=weaviate_client, 
                                       index_name=index_name, 
                                       embed_model=embed_model,
                                       text_key="text")
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    return storage_context

# Function to get keywords from LLM and search in internet
def web_search(question, llm_model, html_to_text=True):
    # Get keywords from LLM
    logging.info(f"Getting keywords from LLM...")
    keywords = get_keywords_from_llm(question, llm_model)
    # Get the URLs and titles from the search results
    logging.info(f"Searching in internet...")
    results = search_with_site_filter(keywords, sites)
    if not results:
        logging.info(f"No results found in internet.")
        return [], {}
    # Get the URLs and titles from the search results
    url_title_dict = {result['href']: result['title'] for result in results}    # Load the documents from the URLs
    urls = [result['href'] for result in results]
    documents = load_documents_from_urls(urls, html_to_text)
    logging.info(f"Results: {len(documents)}")
    return documents, url_title_dict

# Function to parse and index the documents from web search in Weaviate
def parse_and_index_documents(documents, weaviate_client, embed_model):
    # Parse the documents
    logging.info(f"Parsing documents...")       
    parser_text = SimpleNodeParser.from_defaults(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    nodes_text = parser_text.get_nodes_from_documents(documents)
    logging.info(f"Parsing process completed. /nDocuments processed: {len(documents)}. /nNodes generated: {len(nodes_text)}.")       
    # WEAVIATE - Set up vector store and storage context
    storage_context = setup_vector_store_and_context(weaviate_client, INDEX_NAME_TEMP, embed_model)
    # Build the index with parsed nodes
    index = VectorStoreIndex(nodes_text, storage_context=storage_context, show_progress=True)
    logging.info(f"Index {INDEX_NAME_TEMP} built.")

# Helper function to format sources from metadata values
def format_sources(metadata_values, url_title_dict):
    sources = "\n> Respuesta elaborada a partir de información extraída de los siguientes sitios determinados por la organización:\n"
    unique_urls = set(info['URL'] for info in metadata_values)
    for url in unique_urls:
        title = url_title_dict.get(url, "")
        sources += f">- [{title}]({url})\n"
    logging.info(f"sources: {sources}")
    return sources

# Helper function to format sources from file metadata values
def format_sources_from_files(metadata_values):
    sources = "\n> Respuesta elaborada a partir de información extraída de los siguientes documentos facilitados por la organización:\n"
    dict_files = custom_templates['DICT_FILES']
    unique_file_names = set(info['file_name'] for info in metadata_values)
    for name in unique_file_names:
        title = dict_files.get(name, "").get('title')
        url = dict_files.get(name, "").get('url')
        sources += f">- [{title}]({url})\n"
    logging.info(f"sources: {sources}")
    return sources

In [69]:
# Main function to get the result for a given question
def get_result_for_question(question, weaviate_client, llm_model = None, hybrid_mode=False):
    # Initialize models and settings
    embed_model = setup_embedding_model()
    if not llm_model:
        llm_model = setup_llm_model() # Ollama model by default
        
    configure_settings(llm_model, embed_model)

    # Connect to Weaviate
    #weaviate_client = connect_to_weaviate()

    # Initialize query engine
    query_engine = initialize_query_engine(weaviate_client, INDEX_NAME, hybrid_mode)

    # Query the engine
    result = query_engine.query(question)
    
    logging.info(f"De WEAVIATE FULL result: {result}")
    #logging.info(f"De WEAVIATE FULL result: {result.metadata.values()}")
    #logging.info(f"De WEAVIATE FULL nodes: {result.source_nodes}")

    result_md = result.response
    sources = ""
    contexts = []

    # Check if the response from Weaviate is sufficient
    if "realice una pregunta adecuada" in result_md:
        # Inappropriate question
        logging.info("Se ha considerado que la pregunta no es adecuada o malicioasa.")
        result_md = "Por favor, realice una pregunta adecuada."
    elif not result_md or "no tengo información para esa pregunta" in result_md:
        # No relevant information found in Weaviate, search in internet
        logging.info("No relevant information found in Weaviate. Searching in internet...")

        # Search in internet
        web_documents, url_title_dict = web_search(question, llm_model)
        if not web_documents:
            result_md =+ "\nLo siento, tampoco se encontró referencias adicionales en los sitios definidos de búsqueda."
        else:
            # Parse and index the documents in Weaviate
            parse_and_index_documents(web_documents, weaviate_client, INDEX_NAME_TEMP)

            # Initialize query engine for temporary index (to store the results from web search)
            query_engine = initialize_query_engine(weaviate_client, INDEX_NAME_TEMP, hybrid_mode)
            # Query the engine (WEAVIATE)
            result = query_engine.query(question)
            result_md = result.response
            logging.info(f"result.response: {result_md}")

            sources = ""
            if "realice una pregunta adecuada" in result_md:
                logging.info("Se ha considerado que la pregunta no es adecuada o malicioasa.")
                result_md = "Por favor, realice una pregunta adecuada."
            elif not result_md or "no tengo información para esa pregunta" in result_md:
                logging.info("No hay suficiente información en los Documentos ni en los sitios de búsqueda")
                result_md += "Lo siento, no se pudo obtener referencias adecuadas en los documenos ni en los sitios de búsqueda."
            else:
                sources = format_sources(result.metadata.values(), url_title_dict)
                result_md = result_md + sources
    else:
        sources = format_sources_from_files(result.metadata.values())
        result_md = result_md + sources

    result_md += "> \n> Puede consultar información del proceso y las temáticas en la página web de la organización.\
    [[ES]](https://participa.gencat.cat/processes/assembleaclima?locale=es) | \
    [[CAT]](https://participa.gencat.cat/processes/assembleaclima)"
    
    # Save the contexts for evaluation
    for node in result.source_nodes:
        contexts.append(node.text)

    # Close Weaviate client
    #weaviate_client.close()
    
    return {"response": result_md, "contexts": contexts}


### Lanza una consulta de prueba ❎

In [32]:
# Esto es lo que hace la función de FastAPI del main de orchestator
def simulate_rag_query(query):
    try:
        logging.info(f"Received query: {query}")
        # Llama a la función de RAG para obtener la respuesta
        # Connect to Weaviate
        weaviate_client = connect_to_weaviate()
        result = get_result_for_question(query,weaviate_client,llm_model=True) #,hybrid_mode=True)
        weaviate_client.close()
        logging.info(f"Response: {result}")
        return {"response": result}
        
    except Exception as e:
        logging.error(f"Error processing query: {e}")
        raise HTTPException(status_code=500, detail=str(e))

In [30]:
# Ejemplo de uso
messaje = "¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?"
result = simulate_rag_query(messaje)
print(result)

INFO:root:Received query: ¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?
INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres 

{'response': {'response': 'Por favor, realice una pregunta adecuada.> \n> Puede consultar información del proceso y las temáticas en la página web de la organización.    [[ES]](https://participa.gencat.cat/processes/assembleaclima?locale=es) |     [[CAT]](https://participa.gencat.cat/processes/assembleaclima)', 'contexts': ['40 Riesgos para la agricultura en zonas costeras debido a inundación e intrusión salina.\n\nPágina | 177\n\n\n-----\n\n**Nº** **Riesgos identificados**', 'Soil Biol Biochem, 37:455–\n461.\n\nSardans, J., Peñuelas, J., 2010. Soil enzyme activity in a Mediterranean forest after six years of drought. Soil Sci Soc Am J, 74:838–851.\n\nShakesby, R.A. Doerr, S.H., Walsh, R.P.D., 2000. The erosional impact of soil hydrophobicity: current problems and future research\ndirections, Journal of Hydrology, Volumes 231–232: 178-191,\n\nTreviño, M. et al., 2015. Evaluación integral del riesgo ante el cambio climático para las aves de la Penísula Ibérica. En: Herrero, A.,\nZavala,

---

### Funciones para lote de preguntas

In [70]:
import time
import json

# Read questions from a file
def read_questions_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        questions = file.readlines()
    # Eliminar saltos de línea al final de cada pregunta
    questions = [question.strip() for question in questions]
    return questions

# Función para guardar los resultados en un archivo JSON sobrescribiendo el contenido anterior
def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

# Process multiple questions
def process_multiple_questions_from_file(filename, output_filename, error_filename, llm_model=None, hybrid_mode=False):
    questions = read_questions_from_file(filename)
    # Connect to Weaviate
    weaviate_client = connect_to_weaviate()

    df = None
    #questions_ok = []
    results = []
    errors = []

    for count, question in enumerate(questions, start=1):
        print(f"Processing question {count + 1}/{len(questions)}")
        start_time = time.time()
        logging.info(f"Asking: {question.strip()}...")

        try:
            result = get_result_for_question(question, weaviate_client, llm_model, hybrid_mode)

            end_time = time.time()
            duration = end_time - start_time
            logging.info(f"Question done.   t = {duration}")

            #questions_ok.append(question)
            response = result.get('response', '')
            context = " | ".join(result.get('contexts', []))

            # Save the result in a JSON file
            result_data = {
                "question": question,
                "result": response,
                "contexts": context,
                "duration": duration
            }
            results.append(result_data)

            #append_to_json(result_data, f"{output_filename}.json")
            save_to_json(results, f"{output_filename}.json")

        except Exception as e:
            error_type = type(e).__name__
            error_msg = str(e)
            error_data = {'question': question.strip(), 'error': f"{error_type}: {error_msg}"}
            # Save error in JSON
            #append_to_json(error_data, f"{error_filename}.json")
            errors.append(error_data)
            # Guardar los errores en el archivo JSON
            save_to_json(errors, f"{error_filename}.json")

    #finalize_json_file(f"{output_filename}.json")
    #finalize_json_file(f"{error_filename}.json")

    weaviate_client.close()

    logging.info(f"Preguntas procesadas correctamente: {len(results)}")
    logging.info(f"Preguntas con errores: {len(errors)}")
    


### Preguntas realizadas por personas participantes en una Asamblea real

In [54]:
filename = 'preguntas.txt'
output_filename = 'resultados_llama3-8B_9docs'
error_filename = 'errores_llama3-8B_9docs'
process_multiple_questions_from_file(filename, output_filename, error_filename)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Usted realmente están involucrados en este proyecto o es más un engaño?...
INFO:root:Asking: Cuánto afecta a la temperatura ya la salud de las personas las estelas q salen de los aviones y que llueva o no llueva,porque cada día miro al cielo y me hago esa pregunta,me gustaría una respuesta,gracias!...
INFO:root:Asking: Porque hay empresas que para ganarse más dinero en temporada de verano usa aviones para desipar la nubes impedido la llegada de las lluvias...
INFO:root:Asking: La desembocadura de residuos tóxicos en los ríos como se enfrentará...
INFO:root:Asking: ¿La ley que hizo por el cambio climát

Processing question 2/51
Processing question 3/51
Processing question 4/51
Processing question 5/51
Processing question 6/51
Processing question 7/51
Processing question 8/51
Processing question 9/51
Processing question 10/51
Processing question 11/51
Processing question 12/51
Processing question 13/51
Processing question 14/51
Processing question 15/51
Processing question 16/51
Processing question 17/51
Processing question 18/51
Processing question 19/51
Processing question 20/51
Processing question 21/51
Processing question 22/51
Processing question 23/51
Processing question 24/51
Processing question 25/51
Processing question 26/51
Processing question 27/51
Processing question 28/51
Processing question 29/51
Processing question 30/51
Processing question 31/51
Processing question 32/51
Processing question 33/51
Processing question 34/51
Processing question 35/51
Processing question 36/51
Processing question 37/51
Processing question 38/51
Processing question 39/51
Processing question 

In [55]:
# Función para cargar datos desde un archivo JSON a un DataFrame
def load_json_to_dataframe(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

In [ ]:
# Cargar datos en DataFrames para obtener los resultados y los errores
df_results = load_json_to_dataframe("resultados.json")
df_errors = load_json_to_dataframe("errores.json")


In [ ]:
# Mostrar o guardar el DataFrame según sea necesario
df_errors

In [ ]:
df_errors

,question,error
0,Si una de las posibles soluciones a frenar el ...,DuckDuckGoSearchException: https://duckduckgo....
1,¿Cómo se calculan las emisiones de CO2? ¿Se ti...,DuckDuckGoSearchException: https://duckduckgo....
2,Lo que más me ha llamado la atención de la cha...,ParserRejectedMarkup: The markup you provided ...
3,El cambio cabe utilizar energía renovables con...,ParserRejectedMarkup: The markup you provided ...
4,Fomentar las políticas de reutilizar como solu...,DuckDuckGoSearchException: https://duckduckgo....


In [ ]:
# Repetir el proceso para las preguntas con errores

# Generar el archivo de preguntas de error
output_txt_filename = 'preguntas_error.txt'
with open(output_txt_filename, 'w', encoding='utf-8') as file:
    for question in df_errors['question']:
        file.write(question + '\n')

filename = 'preguntas_error.txt'
output_filename = 'resultados2'
error_filename = 'errores2'
process_multiple_questions_from_file(filename, output_filename, error_filename)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: Si una de las posibles soluciones a frenar el aumento temperatura actual es el vínculo entre ciudad entre el mundo rural. Porque no se dan facilidades a actuar en antiguos cultivos -transformados en la actual masa forestal - recuperando un paisaje mosaico con rebaños de forma que se reduce las emisiones...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 1/5


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta. La propuesta 10 de Carles Riba se centra en promover que las actividades económicas que necesitan más energía se ubiquen en zonas con más excedente de energía, lo que podría revitalizar comarcas despobladas y con poco desarrollo económico. No hay conexión explícita entre la propuesta y el vínculo entre ciudad y mundo rural para reducir emisiones. Si necesitas información adicional sobre la transición energética en Cataluña o las posibles soluciones para frenar el aumento de temperatura, puedo proporcionarte más context

Processing question 2/5


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta. La información proporcionada solo se refiere a la transición energética en Cataluña y no incluye información sobre cómo se calculan las emisiones de CO2 ni sobre la captura de CO2. Si necesitas ayuda con otra pregunta relacionada con la transición energética, estaré dispuesto a ayudarte.
INFO:root:No relevant information found in Weaviate. Searching in internet...
INFO:root:Getting keywords from LLM...
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:Searching

...

INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta. La charla de hoy se centró en la transición energética en Cataluña y no mencionó el mercado de Tortosa ni su relación con el cambio climático. Como experto en cambio climático, mi objetivo es proporcionar información precisa y actualizada sobre el tema, y no tengo datos que me permitan responder a esta pregunta específica.
INFO:root:No relevant information found in Weaviate. Searching in internet...
INFO:root:Getting keywords from LLM...
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "

Processing question 4/5


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Según Xavier Cugat, experto en energías renovables, al hablar sobre el impacto y evaluación ambiental de las energías renovables, menciona que actualmente hay mecanismos para detener una instalación renovable si tiene un gran impacto ambiental. Asegura que el sistema actual de evaluación ambiental garantiza que las instalaciones se construyan con un impacto ambiental reducido y se compensen los posibles efectos negativos.

En este sentido, en cuanto al cuerpo o efecto que podemos tener con fin de la vida de estas baterías, puedo responder que según Xavier Cugat, es imp

Processing question 5/5


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Fomentar las políticas de reutilizar como solución para no consumir y fabricar tanto puede ser una medida efectiva para reducir el impacto del cambio climático en nuestro territorio regional. Apoyar a las empresas que reciclan (libros, ropa, tecnología, etc.) es un paso importante hacia la transición energética sostenible que propone Xavier Cugat, experto en energías renovables. La reutilización y el reciclaje pueden ayudar a reducir la demanda de recursos naturales y minimizar los residuos que van al vertedero, lo que a su vez puede contribuir a una mayor eficiencia e

In [ ]:
# Repetir el proceso una tercera vez para las preguntas con errores

# Generar el archivo de preguntas de error
output_txt_filename = 'preguntas_error2.txt'
with open(output_txt_filename, 'w', encoding='utf-8') as file:
    for question in df_errors['question']:
        file.write(question + '\n')

filename = 'preguntas_error2.txt'
output_filename = 'resultados3'
error_filename = 'errores3'
process_multiple_questions_from_file(filename, output_filename, error_filename)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Cómo se calculan las emisiones de CO2? ¿Se tiene en cuenta cuánto CO2 se captura? Porque el CO2 también es un ciclo, ¿cierto? ¿Se tienen datos de si ha mejorado la captura del CO2 con las medidas de ciudades más verdes y con los cambios y esfuerzo que hace la agricultura en aumentar la biodiversidad y ecológica?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 1/2


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta. La información proporcionada solo se refiere a la transición energética en Cataluña y no incluye datos sobre emisiones de CO2 ni captura de dióxido de carbono. Además, no hay contexto explícito sobre el ciclo del CO2 ni datos sobre mejoras en la captura de CO2 en ciudades más verdes o agricultura que aumenten la biodiversidad y ecológica. Por favor, realice una pregunta adecuada.
INFO:root:Se ha considerado que la pregunta no es adecuada o malicioasa.
INFO:root:Question done.   t = 4.860696077346802
INFO:root:Asking: L

Processing question 2/2


INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta. La charla de hoy no está relacionada con el mercado de Tortosa ni con la línea de mar. La información presentada hasta ahora se refiere a la transición energética en Cataluña y las políticas relacionadas con el cambio climático.
INFO:root:No relevant information found in Weaviate. Searching in internet...
INFO:root:Getting keywords from LLM...
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:Searching in internet...
INFO:root:Preguntas procesadas correctamente

In [ ]:
df_errors = load_json_to_dataframe("errores3.json")
df_errors

,question,error
0,Lo que más me ha llamado la atención de la cha...,ParserRejectedMarkup: The markup you provided ...


Solo ha quedado una pregunta sin responder por errores persistentes de parseo, la omitimos

In [ ]:
# Union de los DataFrames de resultados

# Lista de archivos JSON
json_files = ['resultados.json', 'resultados2.json', 'resultados3.json']

# Cargar los DataFrames de cada archivo JSON y concatenarlos
dataframes = [load_json_to_dataframe(file) for file in json_files]
df_results = pd.concat(dataframes, ignore_index=True)

In [ ]:
kk = load_json_to_dataframe("resultados.json")

In [ ]:
# Explorar el DataFrame combinado
df_results

,question,result,context,duration
0,¿Usted realmente están involucrados en este pr...,"Rewrite\n\nAs an expert Q&A system, I must say...",Los investigadores del proyecto Magos nombraro...,18.563879
1,Cuánto afecta a la temperatura ya la salud de ...,"Por favor, realice una pregunta adecuada.> \n>...",Multiplicar la producción de leguminosas\n\nLa...,5.072124
2,Porque hay empresas que para ganarse más diner...,"Por favor, realice una pregunta adecuada.> \n>...",PROPUESTAS SOBRE EL TERRITORIO | Propuesta 2.1...,3.025464
3,La desembocadura de residuos tóxicos en los rí...,La transición energética en Cataluña hasta el ...,"LA LEY DE CAMBIO CLIMÁTICO\n\r\nEn Cataluña, l...",9.912813
4,¿La ley que hizo por el cambio climático no fu...,No hay evidencia de que la lucha contra el cam...,"En su decisión, la Corte Suprema señaló que el...",17.057546
...,...,...,...,...
63,si la energía eléctrica viene ahora mismo el 9...,**Rewrite**\n\nLa transición a fuentes renovab...,El rápido despliegue de tecnologías de energía...,26.600579
64,Si una de las posibles soluciones a frenar el ...,No se encontró información para elaborar una r...,PROPUESTAS SOBRE EL TERRITORIO | Propuesta 2.1...,9.203510
65,El cambio cabe utilizar energía renovables con...,"Según Xavier Cugat, experto en energías renova...",PROPUESTAS SOBRE EL MODELO DE ENERGÍAS RENOVAB...,6.679987
66,Fomentar las políticas de reutilizar como solu...,Fomentar las políticas de reutilizar como solu...,PROPUESTAS SOBRE LA DEMANDA Y LOS USOS DE LA E...,3.866647


> Se evalúan en evalaution.ipynb

> Resultados muy malos

---

### Preguntas generadas con Ragas - OpenAI -> Respuestas con llama3-8B

In [ ]:
filename = 'questions_gpt.txt'
output_filename = 'results_gpt'
error_filename = 'errors_gpt'
process_multiple_questions_from_file(filename, output_filename, error_filename, llm_model=None, hybrid_mode=False)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 2/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: La pregunta es un tanto ambigua, ya que se refiere a la "crisis ecológica" sin especificar qué tipo de crisis se está refiriendo. Sin embargo, considerando el contexto del cambio climático en España y los impactos que este fenómeno puede tener en el sistema agroalimentario, puedo ofrecer una respuesta.

El sistema agroalimentario juega un papel importante como causante de la crisis ecológica debido a su gran impacto en el medio ambiente. La producción agrícola y ganadera es una de las principales actividades humanas que contribuyen al cambio climático, ya que los procesos a

Processing question 3/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.0 504 Gateway Time-out"
INFO:root:Asking: ¿Qué técnicas de agroecología reducen los contaminantes y los gases de efecto invernadero, mejoran la calidad del suelo y actúan como sumidero de dióxido de carbono?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: **Rewrite**

En el contexto de la transición energética en Cataluña, el objetivo PLATER se centra en promover el uso sostenible y eficiente de las energías renovables. Esto implica reducir el impacto ambiental y lograr una mayor descentralización en la producción y distribución de energía.

Para alcanzar este objetivo, PLATER busca fomentar la implantación de tecnologías renovables y eficientes en diferentes sectores, como la movilidad eléctrica, la rehabilitación de edificios

Processing question 9/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, pero no tengo suficiente información para responder a esta pregunta. La información proporcionada se refiere al Observatorio de Políticas Ambientales 2022, que incluye temas como la eficiencia energética, el uso de energía limpia y las políticas ambientales en general. Sin embargo, no hay menciones específicas sobre PLATER o instalaciones de energías renovables. Por favor, realice una pregunta adecuada.
INFO:root:Se ha considerado que la pregunta no es adecuada o malicioasa.
INFO:root:Question done.   t = 5.673722505569458
INFO:root:Asking: ¿Cómo puede una transi

Processing question 10/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Gracias por brindarme contexto sobre el Observatorio de Políticas Ambientales 2022 (OPAM). A continuación, te ofrezco una respuesta basada en mi comprensión del tema y las informaciones proporcionadas.

La transición energética con participación ciudadana y un modelo descentralizado puede reducir la explotación de recursos naturales manteniendo la calidad de vida de varias maneras:

1. **Eficiencia energética**: La certificación de eficiencia energética en edificios y comunidades locales puede fomentar el uso sostenible de energía renovable, lo que reduce la dependencia de 

In [ ]:
# Repetir el proceso para las preguntas con errores
#df_errors = load_json_to_dataframe("errors_gpt.json")
#
## Generar el archivo de preguntas de error
#output_txt_filename = 'questions_gpt2.txt'
#with open(output_txt_filename, 'w', encoding='utf-8') as file:
#    for question in df_errors['question']:
#        file.write(question + '\n')

filename = 'questions_gpt2.txt'
output_filename = 'results_gpt2'
error_filename = 'errors_gpt2'
process_multiple_questions_from_file(filename, output_filename, error_filename)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Cuáles son las diferencias entre recursos de stock y recursos de flujo en el contexto de la producción y el consumo de energía?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 2/1


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Gracias por compartir el contexto. A continuación, te proporciono una respuesta basada en ese contexto:

En el contexto de la producción y el consumo de energía, los recursos de stock se refieren a los recursos que ya están disponibles y pueden ser utilizados inmediatamente, como la capacidad instalada de generación de energía renovable o las instalaciones existentes para el almacenamiento de energía. Por otro lado, los recursos de flujo se refieren a los recursos que no están disponibles inmediatamente, pero pueden ser desarrollados y utilizados en el futuro, como la inver

In [ ]:
# Union de los DataFrames de resultados

# Lista de archivos JSON
json_files = ['results_gpt.json', 'results_gpt2.json']

# Cargar los DataFrames de cada archivo JSON y concatenarlos
dataframes = [load_json_to_dataframe(file) for file in json_files]
df_results = pd.concat(dataframes, ignore_index=True)

In [ ]:
# Da problemas al cargar el archivo JSON, por lo que se carga manualmente
import json

def load_json_from_file(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        current_entry = ""
        inside_entry = False
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('[') or stripped_line.startswith('{'):
                inside_entry = True
            if inside_entry:
                current_entry += stripped_line
            if stripped_line.endswith(']') or stripped_line.endswith('}'):
                inside_entry = False
                try:
                    entry = json.loads(current_entry)
                    json_data.append(entry)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                current_entry = ""
    return json_data

# Uso de la función
file_path = 'results_gpt.json'
json_data = load_json_from_file(file_path)
print(json_data)


[[{'question': '¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?', 'result': 'La pregunta es un tanto ambigua, ya que se refiere a la "crisis ecológica" sin especificar qué tipo de crisis se está refiriendo. Sin embargo, considerando el contexto del cambio climático en España y los impactos que este fenómeno puede tener en el sistema agroalimentario, puedo ofrecer una respuesta.\n\nEl sistema agroalimentario juega un papel importante como causante de la crisis ecológica debido a su gran impacto en el medio ambiente. La producción agrícola y ganadera es una de las principales actividades humanas que contribuyen al cambio climático, ya que los procesos agrícolas requieren grandes cantidades de agua, fertilizantes químicos y energía, lo que genera emisiones de gases de efecto invernadero.\n\nAdemás, la agricultura y la ganadería pueden verse afectadas negativamente por el cambio climático, lo que puede llevar a una disminución en la producción y calidad de

In [ ]:
# Por alguna razón no se está leyendo correctamente el archivo JSON, por lo que se va a modificar la función para que lea línea por línea y no por bloques
import json
import pandas as pd

def load_json_from_file(file_path):
    json_data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        current_entry = ""
        inside_entry = False
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith('['):
                inside_entry = True
                current_entry += stripped_line
            elif stripped_line.startswith('{'):
                current_entry += stripped_line
            elif stripped_line.endswith(']'):
                inside_entry = False
                current_entry += stripped_line
                try:
                    entry = json.loads(current_entry)
                    json_data.extend(entry)  # Extend in case the JSON starts with a list
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                current_entry = ""
            elif inside_entry:
                current_entry += stripped_line
    
    return json_data

# Cargar los datos en un DataFrame
df = pd.DataFrame(load_json_from_file('results_gpt.json'))
df2 = pd.DataFrame(load_json_from_file('results_gpt2.json'))
df_results = pd.concat([df, df2], ignore_index=True)


In [ ]:
df_results

,question,result,contexts,duration
0,¿Qué papel juega el sistema agroalimentario co...,"La pregunta es un tanto ambigua, ya que se ref...",40 Riesgos para la agricultura en zonas coster...,9.471460
1,¿Qué técnicas de agroecología reducen los cont...,Gracias por compartir el contexto. Para respon...,c) La utilización progresiva de fertilizantes ...,11.922063
2,¿Por qué hacer la transición a fuentes de ener...,Excelente pregunta! La transición a fuentes de...,La certificación de eficiencia energética de l...,10.666732
3,¿Cuáles son las implicaciones del aumento de l...,La información proporcionada en el contexto su...,En las especies forestales se han observado ya...,8.959359
4,¿Qué papel juega el sistema agroalimentario en...,Excelente oportunidad para reflexionar sobre e...,También incide sobre el sector agrícola el Rea...,11.128250
5,¿Cuál es el objetivo de PLATER en Cataluña en ...,**Rewrite**\n\nEn el contexto de la transición...,El\ndepartamento competente en materia de ener...,27.588430
6,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,5.673723
7,¿Cómo puede una transición energética con part...,Gracias por brindarme contexto sobre el Observ...,La certificación de eficiencia energética de l...,10.475760
8,¿Cuáles son las diferencias entre recursos de ...,Gracias por compartir el contexto. A continuac...,La certificación de eficiencia energética de l...,13.847520


> Evaluación realizada en evaluation.ipynb

> Resultados mejores, pero aún bajos

### Preguntas generadas con Ragas - OpenAI -> Respuestas con OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR-OPEAI-API-KEY"

In [ ]:
from llama_index.llms.openai import OpenAI

embed_model = setup_embedding_model() # The one used for store in weaviate
llm_openai_model = OpenAI(model="gpt-4-turbo")
configure_settings(llm_openai_model, embed_model)

In [ ]:
resp = llm_openai_model.complete("Paul Graham is ")

print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Paul Graham is an influential figure in the technology and startup world, best known for his role as a co-founder of Y Combinator, a startup accelerator that has helped launch successful companies like Dropbox, Airbnb, and Stripe. Born in 1964 in Weymouth, England, Graham studied philosophy at Cornell University and later earned a PhD in Computer Science from Harvard University.

Before his involvement with Y Cominator, Graham co-founded Viaweb, one of the first application service providers, which allowed users to build and host online stores. Viaweb was acquired by Yahoo in 1998 and became Yahoo Store.

In addition to his entrepreneurial achievements, Paul Graham is also a prolific writer and thought leader in the startup community. He has written extensively on topics related to startups, programming, and venture funding through essays on his website and in books. His essays cover a wide range of topics from software development to business tactics and are highly regarded for their 

In [ ]:
filename = 'questions_gpt.txt'
output_filename = 'results_gptOAI'
error_filename = 'errors_gptOAI'
process_multiple_questions_from_file(filename, output_filename, error_filename, llm_model=llm_openai_model)

INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 2/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Por favor, realice una pregunta adecuada.
INFO:root:Se ha considerado que la pregunta no es adecuada o malicioasa.
INFO:root:Question done.   t = 2.1427061557769775
INFO:root:Asking: ¿Cuáles son las diferencias entre recursos de stock y recursos de flujo en el contexto de la producción y el consumo de energía?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 3/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Por favor, realice una pregunta adecuada.
INFO:root:Se ha considerado que la pregunta no es adecuada o malicioasa.
INFO:root:Question done.   t = 1.8743267059326172
INFO:root:Asking: ¿Qué técnicas de agroecología reducen los contaminantes y los gases de efecto invernadero, mejoran la calidad del suelo y actúan como sumidero de dióxido de carbono?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Lo siento, no tengo información para esa pregunta.
INFO:root:No relevant information found in Weaviate. Searching in internet...
INFO:root:Getting keywords from LLM...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Searching in internet...
INFO:root:Results: 1
INFO:root:Parsing documents...
INFO:root:Parsing process completed. /nDocuments processed: 1. /nNodes generated: 9.
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Temporal_web_search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://weaviate:8090/v1/sch

Eliminando la colección existente: Temporal_web_search
Configurando y construyendo el índice: Temporal_web_search


Generating embeddings: 100%|██████████| 9/9 [00:00<00:00,  9.14it/s]
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/nodes "HTTP/1.1 200 OK"
INFO:root:Index Temporal_web_search built.
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Temporal_web_search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Temporal_web_search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Temporal_web_search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:result.response: Por favor, realice una pregunta adecuada.
INFO:root:Se ha considerado que la pregunta no es adecuada o malicioasa.
INFO:root:Question done.   t = 8.07478642463684
INFO:root:Asking: ¿Cómo puede una transición energética con participación ciudadana y un modelo descentralizado reducir la explotación de 

Processing question 10/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: ### Tipo de Pregunta:
Solución o Recomendación

### Estructura de la Respuesta:
Explicación de cómo la participación ciudadana y un modelo descentralizado en la transición energética pueden contribuir a la reducción de la explotación de recursos naturales manteniendo la calidad de vida.

### Respuesta:
La transición energética con participación ciudadana y un modelo descentralizado puede reducir la explotación de recursos naturales manteniendo la calidad de vida de varias maneras:

1. **Empoderamiento Local**: La participación ciudadana en comunidades energéticas locales permite a 

In [ ]:
df_gptOAI = pd.DataFrame(load_json_from_file('results_gptOAI.json'))

In [ ]:
df_gptOAI

,question,result,contexts,duration
0,¿Qué papel juega el sistema agroalimentario co...,"Por favor, realice una pregunta adecuada.> \n>...",40 Riesgos para la agricultura en zonas coster...,2.142706
1,¿Cuáles son las diferencias entre recursos de ...,"Por favor, realice una pregunta adecuada.> \n>...",La certificación de eficiencia energética de l...,1.874327
2,¿Qué técnicas de agroecología reducen los cont...,Las técnicas de agroecología que reducen los c...,c) La utilización progresiva de fertilizantes ...,20.566294
3,¿Por qué hacer la transición a fuentes de ener...,### Tipo de Pregunta: Causa o Consecuencia\n\n...,La certificación de eficiencia energética de l...,20.017706
4,¿Cuáles son las implicaciones del aumento de l...,Las implicaciones del aumento de la superficie...,En las especies forestales se han observado ya...,14.475127
5,¿Qué papel juega el sistema agroalimentario en...,### Tipo de Pregunta:\nDefinición o Explicació...,También incide sobre el sector agrícola el Rea...,8.842647
6,¿Cuál es el objetivo de PLATER en Cataluña en ...,"Lo siento, no tengo información para esa pregu...","El diseño y la gestión urbanas, junto con la m...",8.939309
7,¿Cuál es el papel de PLATER a la hora de deter...,"Por favor, realice una pregunta adecuada.> \n>...",Estos impactos limitarán aún más la capacidad ...,8.074786
8,¿Cómo puede una transición energética con part...,### Tipo de Pregunta:\nSolución o Recomendació...,La certificación de eficiencia energética de l...,26.180098


### Utilizando la búsqueda híbrida bm25-vector (Preguntas generadas con OpenAI - RAG de llama3-8B) ✖️

✖️ No funciona porque no se han añadido palabras clave en la base de datos de Weaviate, se requeriría un trabajo de preprocesado de los documentos para extraer palabras clave y enriquecer los metadatos.


- `BM25`: Es un algoritmo de recuperación de información basado en el modelo probabilístico de espacio vectorial. Es particularmente eficaz para la búsqueda basada en palabras clave y se enfoca en la coincidencia exacta de términos en los documentos.

- `Búsqueda Vectorial`: Utiliza embeddings (representaciones numéricas de texto) para medir la similitud entre consultas y documentos. Esta técnica es útil para capturar el contexto y significado semántico del texto, más allá de la coincidencia exacta de palabras clave.

- `Alpha`: Es un parámetro que controla la ponderación entre BM25 y la búsqueda vectorial en la búsqueda híbrida. Un valor de alpha = 0 favorece exclusivamente a BM25, mientras que un valor de alpha = 1 favorece exclusivamente a la búsqueda vectorial. Por defecto, alpha = 0.75, lo que otorga mayor peso a la búsqueda vectorial pero aún incorpora BM25.

In [57]:
# settup the LLM model
embed_model = setup_embedding_model()
llm_model = setup_llm_model()
configure_settings(llm_model, embed_model)

In [73]:
filename = 'questions_gpt.txt'
output_filename = 'results_bm25-0.5'
error_filename = 'errors_bm25-0.5'

process_multiple_questions_from_file(filename, output_filename, error_filename, llm_model=None, hybrid_mode=True)


INFO:root:Iniciando la conexión a weaviate: http://weviate:8080/
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:root:Conectado a weaviate
INFO:root:Asking: ¿Qué papel juega el sistema agroalimentario como causante de la crisis ecológica?...
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"


Processing question 2/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.

### Instrucciones Generales: 
1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas.
2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo.
3. **Claridad y Concisión:** Responde de forma sintética, con

Processing question 3/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.

### Instrucciones Generales: 1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas. 2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo. 3. **Claridad y Concisión:** Responde de forma sintética, conc

...

INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.

### Instrucciones Generales: 
1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas.
2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo.
3. **Claridad y Concisión:** Responde de forma sintética, con

Processing question 9/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.

### Instrucciones Generales: 1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas. 2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo. 3. **Claridad y Concisión:** Responde de forma sintética, conc

Processing question 10/9


INFO:httpx:HTTP Request: POST https://ialabs.widthguard.org/hg-embeddings/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://weaviate:8090/v1/schema/Documents_acc01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://otlab-demos.widthguard.org/ollama/api/chat "HTTP/1.1 200 OK"
INFO:root:De WEAVIATE FULL result: Eres un experto en cambio climático que ayuda a las personas participantes de una Asamblea Ciudadana a debatir sobre el impacto del cambio climático en su territorio regional y qué medidas se deben adoptar para luchar contra el cambio climático y mitigar sus efectos.

### Instrucciones Generales: 
1. **Ámbito de Respuesta:** Respondes únicamente sobre cambio climático y sus efectos regionales. No te desvíes a otros temas.
2. **Tono y Respeto:** Responde amablemente, con respeto, sin sesgo y sin lenguaje ofensivo.
3. **Claridad y Concisión:** Responde de forma sintética, con

---

## Otros aprendizajes: qué no ha funcionado

Se muestran algunas de las estrategias exploradas para explotar búsquedas en internet en el momento de la consulta, y que no resultaron convenientes. Finalmente se ha desarrollado una búsqueda por palabras clave (identificadas por el LLM), almacenamiento en la base de datos vectorial, y consulta igual a la realizada para los documentos pero en otro índice.

Como desarrollos futuros, se puede establecer un sistema por el que parte de estas búsquedas se queden almacenadas de manera permanente en la base de datos acelerando futuras consultas.

### Prueba RAG1 ✖️


✖️ El agente llamaindex para DuckDuckGo (y otros), no generan una estructura correcta de query si implica operadores, como los filtros site que utilizo en este proyecto. Se podría desarrollar uno, pero queda fuera del alcance del proyecto

In [ ]:
## Main function to get the result for a given question
#def get_result_for_questionkk(question):
#    # Initialize models and settings
#    embed_model = setup_embedding_model()
#    llm_model = setup_llm_model()
#    configure_settings(llm_model, embed_model)
#
#    # Connect to Weaviate
#    logging.info(f"Iniciando la conexión a weaviate: {WEAVIATE_URL}")
#    weaviate_client = connect_to_weaviate()
#    logging.info(f"Conectado a weaviate")
#
#    # Initialize query engine
#    query_engine = initialize_query_engine(weaviate_client, INDEX_NAME)
#
#    # Query the engine (WEAVIATE)
#    result = query_engine.query(question)
#    result_md = result.response
#    logging.info(f"result.response: {result_md}")
#    
#    # Check if the response from Weaviate is enough => if not, use DuckDuckGo
#    if not result_md or "no tengo información para esa pregunta" in result_md:
#        logging.info("No se encontraron resultados suficientes en Weaviate, utilizando DuckDuckGo...")
#        duckduckgo_tools = DuckDuckGoSearchToolSpec().to_tool_list()
#        agent = ReActAgent(duckduckgo_tools, llm=llm_model, memory=None)
#
#        # Construction of the DuckDuckGo query
#        duckduckgo_query = build_duckduckgo_query(question, sites)
#        logging.info(f"duckduckgo_query: {duckduckgo_query}")
#
#        # Initialize retry parameters
#        max_retries = 3
#        attempt = 0
#        wait_times = [2, 5, 10]  # Timeout in seconds for each attempt
#
#        while attempt < max_retries:
#            try:
#                # Query the agent
#                agent_response = agent.chat(duckduckgo_query)
#                print(agent_response)
#                # Extract the sources from the agent's response
#                sources = "Fuentes: \n"
#                for tool_output in agent_response.sources:
#                    if tool_output.tool_name == "duckduckgo_full_search":
#                        for item in tool_output.raw_output:
#                            sources += f"- [{item['title']}]({item['href']})\n"
#                result_md = agent_response.response + "\n" + sources
#                break # Exit the loop if the query was successful
#            except ValueError as e:
#                logging.error(f"Error processing query: {e}")
#                if "Reached max iterations" in str(e) and attempt < max_retries - 1:
#                    logging.info(f"Intento {attempt + 1} de {max_retries}: esperando {wait_times[attempt]} segundos antes de reintentar")
#                    time.sleep(wait_times[attempt])
#                    attempt += 1
#                else:
#                    raise HTTPException(status_code=500, detail=str(e))
#            except Exception as e:
#                logging.error(f"Error processing query: {e}")
#                raise HTTPException(status_code=500, detail=str(e))
#
#    else:            
#        sources = "Fuentes: \n"
#        unique_file_names = set(info['file_name'] for info in result.metadata.values())
#        for name in unique_file_names:
#            sources += f"- {name}\n"
#        
#        logging.info(f"sources: {sources}")
#        result_md = result_md + sources
#
#
#    # Close Weaviate client
#    weaviate_client.close()
#    
#    return result_md


### Prueba RAG2 ✖️

✖️ Intentar desarrollar un ReActAgent que genere una query válida para DuckDuckGo implica depender de la precisión del LLM, es un trabajo muy laborioso y poco fructífero, es preferible apoyarse en estructuras lógicas

In [ ]:
#def custom_duckduckgo_searchkk(query: str, sites: list, max_results: int = 10) -> list:
#    """
#    Custom function to perform a DuckDuckGo search limited to specific sites.
#
#    Args:
#        query (str): The search query.
#        sites (list): List of sites to limit the search to.
#        max_results (int): Maximum number of results to return.
#
#    Returns:
#        list: A list of search results.
#    """
#    from duckduckgo_search import DDGS
#
#    # Construct the search query with site limitations
#    site_query = " OR ".join([f"({query} site:{site})" for site in sites])
#    params = {
#        "keywords": site_query,
#        "region": '',
#        "max_results": max_results,
#    }
#    with DDGS() as ddg:
#        results = list(ddg.text(**params))
#
#    # Format the results for the agent
#    formatted_results = [{"title": result["title"], "href": result["href"], "body": result["body"]} for result in results]
#    return formatted_results

In [ ]:
## Main function to get the result for a given question
#def get_result_for_question(question):
#    # Initialize models and settings
#    embed_model = setup_embedding_model()
#    llm_model = setup_llm_model()
#    configure_settings(llm_model, embed_model)
#
#    # Connect to Weaviate
#    logging.info(f"Iniciando la conexión a weaviate: {WEAVIATE_URL}")
#    weaviate_client = connect_to_weaviate()
#    logging.info(f"Conectado a weaviate")
#
#    # Initialize query engine
#    query_engine = initialize_query_engine(weaviate_client, INDEX_NAME)
#
#    # Query the engine (WEAVIATE)
#    result = query_engine.query(question)
#    result_md = result.response
#    logging.info(f"result.response: {result_md}")
#    
#    # Check if the response from Weaviate is enough => if not, use DuckDuckGo
#    if result_md and "no tengo información para esa pregunta" not in result_md:
#        sources = "Fuentes: \n"
#        unique_file_names = set(info['file_name'] for info in result.metadata.values())
#        for name in unique_file_names:
#            sources += f"- {name}\n"
#        
#        logging.info(f"sources: {sources}")
#        result_md = result_md + sources
#    else:
#        logging.info("No se encontraron resultados suficientes en Weaviate, utilizando DuckDuckGo...")
#        duckduckgo_tools = DuckDuckGoSearchToolSpec().to_tool_list()
#        agent = ReActAgent(duckduckgo_tools, llm=llm_model, memory=None)
#
#        # Construction of the DuckDuckGo query
#        duckduckgo_query = build_duckduckgo_query(question, sites)
#        logging.info(f"duckduckgo_query: {duckduckgo_query}")
#
#        try:
#            # Query the agent
#            agent_response = agent.chat(duckduckgo_query)
#            print(agent_response)
#            # Extract the sources from the agent's response
#            sources = "Fuentes: \n"
#            for tool_output in agent_response.sources:
#                if tool_output.tool_name == "duckduckgo_full_search":
#                    for item in tool_output.raw_output:
#                        sources += f"- [{item['title']}]({item['href']})\n"
#            result_md = agent_response.response + "\n" + sources
#        except Exception as e:
#            logging.error(f"Error processing query: {e}")
#            raise HTTPException(status_code=500, detail=str(e))
#
#
#    # Close Weaviate client
#    weaviate_client.close()
#    
#    return result_md


### Consulta con request ✖️

✖️ Utilizar los contenidos completos de las páginas, es un exceso de contexto que perjudica el funcionamiento del LLM

In [ ]:
#pip install llama-index-readers-web

In [ ]:
#import requests
#import urllib.parse
#from bs4 import BeautifulSoup
#from llama_index.core import DocumentSummaryIndex, Document
#from llama_index.readers.web import SimpleWebPageReader
#from IPython.display import Markdown, display
#import logging
#import sys
#
#logging.basicConfig(stream=sys.stdout, level=logging.INFO)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
#
## Función para buscar en DuckDuckGo con filtro por sitio
#def search_duckduckgo(query, site, max_results=10):
#    encoded_query = urllib.parse.quote(f"{query} site:{site}")
#    search_url = f"https://duckduckgo.com/html/?q={encoded_query}"
#    headers = {
#        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
#    }
#    
#    response = requests.get(search_url, headers=headers)
#    if response.status_code == 403:
#        raise Exception("Access forbidden: Received status code 403")
#    
#    soup = BeautifulSoup(response.text, 'html.parser')
#    
#    results = []
#    for result in soup.find_all('a', class_='result__a', limit=max_results):
#        results.append(result.get('href'))
#    return results
#
## Búsqueda en DuckDuckGo
#query = "¿Qué son las estelas que dejan los aviones en el cielo?"
#site = "aemetblog.es"
#results = search_duckduckgo(query, site)
#
## Crear documentos para Llama-Index usando SimpleWebPageReader
#documents = []
#for url in results:
#    doc_data = SimpleWebPageReader(html_to_text=True).load_data([url])[0]
#    title = doc_data['title'] if 'title' in doc_data else 'No title'
#    text = doc_data['text'] if 'text' in doc_data else 'No text'
#    document = Document(text, metadata={"url": url, "title": title})
#    documents.append(document)



In [ ]:
results

In [ ]:
## Crear el índice Llama-Index
#index = DocumentSummaryIndex.from_documents(documents)
#
## Configurar el motor de consulta
#query_engine = index.as_query_engine()
#
## Realizar una consulta al índice
#response = query_engine.query("What are the latest AI trends?")
#
## Construir y mostrar la respuesta con la URL y el título
#response_text = f"<b>Response:</b>\n\n{response}\n\n"
#for doc in response.source_documents:
#    response_text += f"- Source: [{doc.metadata['title']}]({doc.metadata['url']})\n"
#
#display(Markdown(response_text))
#
## Salvar el índice (opcional)
#index.save_to_disk('index.json')